In [19]:
import pandas as pd
import numpy as np
import os

In [20]:
netflix_StrangerThings = '/home/bde/Data/5Gdataset-master/5G-production-dataset/Netflix/Driving/Season3-StrangerThings/'
netflix_animated_RickandMorty = '/home/bde/Data/5Gdataset-master/5G-production-dataset/Netflix/Driving/animated-RickandMorty/'
amazon_TheExpanse = '/home/bde/Data/5Gdataset-master/5G-production-dataset/Amazon_Prime/Driving/Season3-TheExpanse/'
amazon_animated_AdventureTime = '/home/bde/Data/5Gdataset-master/5G-production-dataset/Amazon_Prime/Driving/animated-AdventureTime/'
download = '/home/bde/Data/5Gdataset-master/5G-production-dataset/Download/Driving/'
df_list = []
file_dir = download

for file in os.listdir(file_dir)[:]:
    print(file)
    df = pd.read_csv(file_dir+file)

    df = df[['Timestamp', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'DL_bitrate', 'UL_bitrate']]
    df = df.replace('-', np.nan)
    
    df['date'] = pd.to_datetime(df['Timestamp'], format="%Y.%m.%d_%H.%M.%S")
    df = df.drop(columns=['Timestamp'])

    df['RSRP'] = pd.to_numeric(df['RSRP'], downcast='float')
    df['RSRQ'] = pd.to_numeric(df['RSRQ'], downcast='float')
    df['SNR'] = pd.to_numeric(df['SNR'], downcast='float')
    df['CQI'] = pd.to_numeric(df['CQI'], downcast='float')
    df['RSSI'] = pd.to_numeric(df['RSSI'], downcast='float')
    df['DL_bitrate'] = pd.to_numeric(df['DL_bitrate'], downcast='float')
    df['UL_bitrate'] = pd.to_numeric(df['UL_bitrate'], downcast='float')

    df.fillna(method='ffill', inplace=True)
    df.fillna('0', inplace=True)

    df.drop_duplicates(subset=['date'], inplace=True)
    df.info()
    #df.info(show_counts=True)
    #df.head()
    #df.plot(subplots=True, layout=(3,3), figsize=(40, 25))
    #df.to_csv('5Gdataset-{}.csv'.format(file), encoding='utf-8', index=False)
    df_list.append(df)

B_2019.12.17_07.32.39.csv
<class 'pandas.core.frame.DataFrame'>
Index: 2295 entries, 0 to 2574
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   RSRP        2295 non-null   float32       
 1   RSRQ        2295 non-null   float32       
 2   SNR         2295 non-null   float32       
 3   CQI         2295 non-null   float32       
 4   RSSI        2295 non-null   float32       
 5   DL_bitrate  2295 non-null   float32       
 6   UL_bitrate  2295 non-null   float32       
 7   date        2295 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float32(7)
memory usage: 98.6 KB
B_2020.01.16_09.56.56.csv
<class 'pandas.core.frame.DataFrame'>
Index: 1536 entries, 0 to 1740
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   RSRP        1536 non-null   float32       
 1   RSRQ        1536 non-null   float32       
 2   SNR      

In [21]:
df = pd.concat(df_list, axis=0)
df.sort_values('date', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24446 entries, 0 to 857
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   RSRP        24446 non-null  float32       
 1   RSRQ        24446 non-null  float32       
 2   SNR         24446 non-null  object        
 3   CQI         24446 non-null  object        
 4   RSSI        24446 non-null  object        
 5   DL_bitrate  24446 non-null  float32       
 6   UL_bitrate  24446 non-null  float32       
 7   date        24446 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float32(4), object(3)
memory usage: 1.3+ MB


In [22]:
df.head()

,RSRP,RSRQ,SNR,CQI,RSSI,DL_bitrate,UL_bitrate,date
0,-99.0,-15.0,6.0,12.0,-90.0,7.0,1.0,2019-12-14 10:16:30
1,-99.0,-15.0,6.0,12.0,-90.0,7.0,1.0,2019-12-14 10:16:31
3,-102.0,-14.0,6.0,12.0,-90.0,0.0,0.0,2019-12-14 10:16:32
4,-102.0,-14.0,6.0,12.0,-90.0,9.0,12.0,2019-12-14 10:16:33
5,-102.0,-14.0,4.0,12.0,-90.0,0.0,0.0,2019-12-14 10:16:34


In [23]:
df = df[['date', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'UL_bitrate', 'DL_bitrate']]

In [24]:
df.to_csv('data/5Gdataset/download.csv', encoding='utf-8', index=False)